In [1]:
import fredapi
import pandas as pd
from _api_keys import get_fred_api_key

fred = fredapi.Fred(api_key=get_fred_api_key())

In [7]:
df = fred.search('unemployment')
df['count'] = df.groupby('title')['title'].transform('count')
df

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes,count
series id,,,,,,,,,,,,,,,,
UNRATE,UNRATE,2023-08-18,2023-08-18,Unemployment Rate,1948-01-01,2023-07-01,Monthly,M,Percent,%,Seasonally Adjusted,SA,2023-08-04 07:45:03-05:00,92,The unemployment rate represents the number of...,2
UNRATENSA,UNRATENSA,2023-08-18,2023-08-18,Unemployment Rate,1948-01-01,2023-07-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-04 07:45:34-05:00,53,The unemployment rate represents the number of...,2
CCSA,CCSA,2023-08-18,2023-08-18,Continued Claims (Insured Unemployment),1967-01-07,2023-08-05,"Weekly, Ending Saturday",W,Number,Number,Seasonally Adjusted,SA,2023-08-17 07:33:03-05:00,74,"Continued claims, also referred to as insured ...",2
CCNSA,CCNSA,2023-08-18,2023-08-18,Continued Claims (Insured Unemployment),1967-01-07,2023-08-05,"Weekly, Ending Saturday",W,Number,Number,Not Seasonally Adjusted,NSA,2023-08-17 07:33:03-05:00,41,"Continued claims, also referred to as insured ...",2
UNEMPLOY,UNEMPLOY,2023-08-18,2023-08-18,Unemployment Level,1948-01-01,2023-07-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2023-08-04 07:45:25-05:00,65,The series comes from the 'Current Population ...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OHMADI7URN,OHMADI7URN,2023-08-18,2023-08-18,"Unemployment Rate in Madison County, OH",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:19:54-05:00,5,These data come from the Current Population Su...,1
ILLAKE7URN,ILLAKE7URN,2023-08-18,2023-08-18,"Unemployment Rate in Lake County, IL",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:19:03-05:00,5,These data come from the Current Population Su...,1
PAADAM0URN,PAADAM0URN,2023-08-18,2023-08-18,"Unemployment Rate in Adams County, PA",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:13:12-05:00,5,These data come from the Current Population Su...,1


In [12]:
df = df.loc[df['title'].str.contains("Unemployment Rate in")]
df = df.loc[df.frequency_short.eq('M')]
df.to_parquet("./fred_unemployment.parquet", index=False)